In [ ]:
# In this file, I will re-run Horlbeck on Parrish due to the following reasons:
# SLKB uses normalised o9unts for Parrish which is not  a correct approach. We need raw counts
# We will use ETP for both PC9 and HeLa and run horlbeck, not plasmid.

%run Horlbeck_Functions.ipynb


In [ ]:
Parrish = pd.read_table("../InputData/Parrish/GSE178179_pgPEN_counts_PC9.txt")

Parrish.rename(columns = {'gRNA1_seq' :'sgRNA_guide_seq_g1', 'gRNA2_seq' :'sgRNA_guide_seq_g2' }, inplace = True)#Parrish
Parrish[['sgRNA_target_name_g1', 'sgRNA_target_name_g2']] = Parrish['paralog_pair'].str.split('|', expand=True)
Parrish['study_origin'] = 'Parrish'
Parrish['target_type'] = 'Dual'
## Anything with nt  is control in this study
Parrish

Parrish.loc[Parrish['sgRNA_target_name_g1'].str.contains('^nt[0-8]{1}'),'sgRNA_target_name_g1'] = 'CONTROL'
Parrish.loc[Parrish['sgRNA_target_name_g2'].str.contains('^nt[0-8]{1}'),'sgRNA_target_name_g2'] = 'CONTROL'

Parrish.loc[(Parrish['sgRNA_target_name_g1'] == Parrish['sgRNA_target_name_g2']), 'target_type'] =   'Single'
Parrish.loc[(Parrish['sgRNA_target_name_g1'] == 'CONTROL') | (Parrish['sgRNA_target_name_g2'] == 'CONTROL'), 'target_type'] = 'Single'
# From paper: Five hundred double non-targeting pgRNAs were included as a control. This
Parrish.loc[(Parrish['sgRNA_target_name_g1'].str.startswith('NTpg')),'sgRNA_target_name_g1'] = "CONTROL" # & (Parrish['sgRNA_target_name_g2'] == 'NA'), 'target_type'] = 'Control'
Parrish.loc[(Parrish['sgRNA_target_name_g1'] == "CONTROL") & (Parrish['sgRNA_target_name_g2'] == "NA") , 'sgRNA_target_name_g2'] = "CONTROL"
Parrish.loc[(Parrish['sgRNA_target_name_g1'] == "CONTROL") & (Parrish['sgRNA_target_name_g2'] == "CONTROL") , 'target_type'] = "Control"

Parrish['gene_pair_id'] = Parrish['sgRNA_target_name_g1'] + "|" + Parrish['sgRNA_target_name_g2']
Parrish['sgRNA_guide_name_g1'] = Parrish['sgRNA_guide_seq_g1'] 
Parrish['sgRNA_guide_name_g2'] = Parrish['sgRNA_guide_seq_g2']

Parrish['gene_pair_orientation'] = Parrish.apply(lambda row: 'A_B' if row['sgRNA_target_name_g1'] > row['sgRNA_target_name_g2'] else 'B_A', axis=1)
display(Parrish)

In [ ]:
Parrish_PC9 = Parrish.copy()
Parrish_PC9.drop(['PC9_plasmid', 'HeLa_plasmid','HeLa_ETP', 'HeLa_LTP_RepA', 'HeLa_LTP_RepB', 'HeLa_LTP_RepC'], inplace = True, axis = 1)
Parrish_PC9['study_conditions'] = "PC9_ETP_RepA;PC9_ETP_RepB;PC9_ETP_RepC;PC9_LTP_RepA;PC9_LTP_RepB;PC9_LTP_RepC" #"T0_1;T0_2;T0_3;T12_1;T12_2;T12_3"
Parrish_PC9['gene_pair'] = Parrish_PC9['gene_pair_id']
Parrish_PC9['count_replicates'] = Parrish_PC9['PC9_ETP_RepA'].astype(str) + ";" + Parrish_PC9['PC9_ETP_RepB'].astype(str) + ";" + Parrish_PC9['PC9_ETP_RepC'].astype(str) + ";" + Parrish_PC9['PC9_LTP_RepA'].astype(str) + ";" + Parrish_PC9['PC9_LTP_RepB'].astype(str) + ";" + Parrish_PC9['PC9_LTP_RepC'].astype(str)
Parrish_PC9['cell_line_origin'] = "PC9"
Parrish_PC9['T0_counts'] = Parrish_PC9['PC9_ETP_RepA'].astype(str) + ";" + Parrish_PC9['PC9_ETP_RepB'].astype(str) + ";" + Parrish_PC9['PC9_ETP_RepC'].astype(str)
Parrish_PC9['T0_replicate_names'] =  'PC9_ETP_RepA;PC9_ETP_RepB;PC9_ETP_RepC'
Parrish_PC9['TEnd_counts'] = Parrish_PC9['PC9_LTP_RepA'].astype(str) + ";" + Parrish_PC9['PC9_LTP_RepB'].astype(str) + ";" + Parrish_PC9['PC9_LTP_RepC'].astype(str)
Parrish_PC9['TEnd_replicate_names'] = "PC9_LTP_RepA;PC9_LTP_RepB;PC9_LTP_RepC"
Parrish_PC9 = Parrish_PC9[['sgRNA_guide_name_g1','sgRNA_guide_name_g2','sgRNA_target_name_g1','sgRNA_target_name_g2','count_replicates','cell_line_origin','study_conditions','study_origin','target_type','T0_counts','T0_replicate_names','TEnd_counts','TEnd_replicate_names','gene_pair','gene_pair_orientation']]
Parrish_PC9

Parrish_Horlbeck_PC9 = run_horlbeck_score(Parrish_PC9, 'Parrish', "PC9", do_preprocessing= True, re_run=True, filterThreshold=0)

s1 = Parrish_Horlbeck_PC9['HORLBECK_SCORE']


In [ ]:

Parrish_HeLa = Parrish.copy()
Parrish_HeLa.drop(['PC9_plasmid', 'HeLa_plasmid','PC9_ETP_RepA','PC9_ETP_RepB','PC9_ETP_RepC', 'PC9_LTP_RepA', 'PC9_LTP_RepB', 'PC9_LTP_RepC'], inplace = True, axis = 1)
Parrish_HeLa['study_conditions'] = "HeLa_ETP;HeLa_LTP_RepA;HeLa_LTP_RepB;HeLa_LTP_RepC" 
Parrish_HeLa['gene_pair'] = Parrish_HeLa['gene_pair_id']
Parrish_HeLa['count_replicates'] = Parrish_HeLa['HeLa_ETP'].astype(str) + ";"+ Parrish_HeLa['HeLa_LTP_RepA'].astype(str) + ";" + Parrish_HeLa['HeLa_LTP_RepB'].astype(str) + ";" + Parrish_HeLa['HeLa_LTP_RepC'].astype(str)
Parrish_HeLa['cell_line_origin'] = "HeLa"
Parrish_HeLa['T0_counts'] = Parrish_HeLa['HeLa_ETP'].astype(str)
Parrish_HeLa['T0_replicate_names'] =  'HeLa_ETP'
Parrish_HeLa['TEnd_counts'] = Parrish_HeLa['HeLa_LTP_RepA'].astype(str) + ";" + Parrish_HeLa['HeLa_LTP_RepB'].astype(str) + ";" + Parrish_HeLa['HeLa_LTP_RepC'].astype(str)
Parrish_HeLa['TEnd_replicate_names'] = "HeLa_LTP_RepA;HeLa_LTP_RepB;HeLa_LTP_RepC"
Parrish_HeLa = Parrish_HeLa[['sgRNA_guide_name_g1','sgRNA_guide_name_g2','sgRNA_target_name_g1','sgRNA_target_name_g2','count_replicates','cell_line_origin','study_conditions','study_origin','target_type','T0_counts','T0_replicate_names','TEnd_counts','TEnd_replicate_names','gene_pair','gene_pair_orientation']]
Parrish_HeLa

Parrish_Horlbeck_HeLa = run_horlbeck_score(Parrish_HeLa, 'Parrish', "HeLa", do_preprocessing= True, re_run=True,filterThreshold=0)
s2 = Parrish_Horlbeck_HeLa['HORLBECK_SCORE']


In [ ]:
# OK so Horlbeck Score was ran on SONIC. Now read and combine the files
Score_PC9 = pd.read_csv("Parrish_Horlbeck_PC9")
Score_HeLa = pd.read_csv("Parrish_Horlbeck_HeLa")


Score_PC9.rename(columns = {'SL_score': 'PC9' }, inplace = True)
Score_HeLa.rename(columns = {'SL_score': 'HeLa' }, inplace = True)

Score_PC9 =  Score_PC9[['gene_pair', 'PC9']]
Score_HeLa =  Score_HeLa[['gene_pair','HeLa']]
Score_PC9.set_index('gene_pair', inplace = True)
Score_HeLa.set_index('gene_pair', inplace = True)

Horlbeck_Parrish = Score_PC9.join(Score_HeLa, how='outer')


In [ ]:
#Horlbeck_Parrish.to_csv("Horlbeck Output/Horlbeck_Parrish.csv")
Score_PC9.to_csv("Horlbeck Output/Horlbeck_Parrish_PC9.csv") 
Score_HeLa.to_csv("Horlbeck Output/Horlbeck_Parrish_HeLa.csv") 

